# Exercice 3 - Liquid Gaz Point from Virial Coefficients


## Q1 - Conditions de criticité

a) L'équation d'état du Viriel est donnée par:
$$
\frac{PV}{Nk_BT}=\sum_{l=1}^{\infty} a_l(\frac{\lambda^3}{v})^{l-1}
$$
où $v=\frac{V}{N}$ et on définit $\rho = \frac{N}{V}$ alors, à l'ordre 3, l'équation devient:
$$
\frac{PV}{N k_B T}
= \sum_{\ell = 1}^{\infty} a_\ell \left( \lambda^3\, \rho \right)^{\ell - 1}
\simeq 
a_1 
+ a_2 \lambda^3 \rho
+ a_3 \lambda^6 \rho^2 \\
\Longleftrightarrow \qquad
\frac{P}{k_B T}
\simeq
a_1 \rho
+ a_2 \lambda^3 \rho^2
+ a_3 \lambda^6 \rho^3
$$

b) La première et seconde dérivée sont:
$$
\partial_{\rho} \mathcal{L} 
= k_{B} T \left( a_{1} + 2 a_{2}\, \lambda^3 \,\rho + 3 a_{3}\,\lambda^6\,\rho^{2} \right) \\

\partial_{\rho}^{2} \mathcal{L}
= k_{B} T \left( 2\, \lambda^3\, a_{2} + 6\,\lambda^3\, a_{3}\,\rho \right)
$$

c) Les deux équations de criticité obtenues sont:
$$
1 + 2\,\lambda^3\, a_{2}(T_c)\,\rho_c + 3\,\lambda^6\, a_{3}(T_c)\,\rho^{2}_c  = 0 \qquad(*) \\
2 a_{2}(T_c) + 6\,\lambda^3\, a_{3}(T_c)\,\rho_c  = 0 \qquad(**)
$$




## Q2 - Condition reliant $a_2(T_c)$ et $a_3(T_c)$

A partir de l'équation $(**)$ on obtient $\rho_c = -\,\frac{2\,a_2(T_c)}{\lambda^3\,6\,a_3(T_c)}$ et en remplaçant cette variable dans $(*)$ nous avons:
$$
1 + 2\lambda^3 a_2(T_c)\!\left(-\,\frac{2 a_2(T_c)}{\lambda^3\,6\,a_3(T_c)}\right)
+ 3\lambda^6 a_3(T_c)\!\left(-\,\frac{2 a_2(T_c)}{\lambda^3\,6\,a_3(T_c)}\right)^{\!2}
= 0\\
\Rightarrow\quad
1 - \frac{2 a_2(T_c)^2}{3 a_3(T_c)}
+ \frac{3\lambda^6 a_3(T_c)\,4 a_2(T_c)^2}{\lambda^6\,36\,a_3(T_c)^2}
= 0\\
\Rightarrow\quad
1 - \frac{2 a_2(T_c)^2}{3 a_3(T_c)}
+ \frac{1}{3}\frac{a_2(T_c)^2}{a_3(T_c)}
= 0\\
\Rightarrow\quad
\frac{a_2(T_c)^2}{3} = a_3(T_c).
$$


## Q3 - Détermination du point critique

a) Nous avons $b_2 = ??$ et $b_3=??$ alors $a_2=-b_2^2= ???$ et $a_3= 4\,b_2^2-2\,b_3 = $